In [ ]:
import numpy as np
import os
import pathlib
import skimage.io as io
import skimage.transform as tf
import skimage.exposure as ex
import skimage.color as color
import torch
import torch.optim as optim

In [ ]:
!pip install haroun==0.0.5

# import my Library (Pytorch Framework)
from haroun import Data
from haroun import Model
from haroun.losses import RMSE

In [ ]:
def flip(images, labels, axis): 
    flipped_images = np.flip(images, axis)
    flipped_labels = labels
    return flipped_images,flipped_labels

def brightness(images, labels, gamma):
    brightness_images =  np.array([ex.adjust_gamma(image, gamma ,gain=1) for image in images])
    brightness_labels = labels
    return brightness_images, brightness_labels


def data_augmentation(images, labels):
    # Data augmentation (flip_horizontal)
    flipped_y_images,flipped_y_labels = flip(images, labels, axis=2)
    
    # Concatenate arrays
    images = np.concatenate([images, flipped_y_images])
    labels = np.concatenate([labels, flipped_y_labels])
    
    darken_images, darken_labels = brightness(images, labels, gamma=1.5)
    brighten_images, brighten_labels = brightness(images, labels, gamma=0.5)
    
    # Concatenate arrays
    images = np.concatenate([images, darken_images, brighten_images])
    labels = np.concatenate([labels, darken_labels, brighten_labels])
    
    return images, labels

In [ ]:
def load_data():
    path = pathlib.Path.cwd().parent / "input" / "covid19-radiography-dataset"
    path = path / "COVID-19_Radiography_Dataset" / "COVID-19_Radiography_Dataset"
    images = []
    labels = []


    for directory in os.listdir(path):
        data_path = path / directory
        for im in os.listdir(data_path)[:]:
            image = io.imread(f"{data_path}/{im}")
            image = color.rgb2gray(image)
            image = tf.resize(image, (64, 64))
            images.append(image)
            labels.append(directory)

    images = np.array(images)
    labels = np.array(labels)
    
    images, labels = data_augmentation(images, labels)

    return images, labels

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classes = {'Normal': 0, 'Lung_Opacity': 1, 'Viral Pneumonia': 2, 'COVID': 3}

data = Data(loader=load_data(), classes=classes)
data.shape()
data.show()

In [ ]:
data.dataset(split_size=0.05, shuffle=True, random_state=42,
             images_format=torch.float32, labels_format=torch.float32,
             permute=True, device=device)

In [ ]:
# Define CNN 
class conv_layer(torch.nn.Module):

    def __init__(self, in_features, out_features):
        super(conv_layer, self).__init__()
        self.conv = torch.nn.Conv2d(in_features, out_features, kernel_size=3, stride=1, padding=1)
        self.relu = torch.nn.ReLU()
        self.norm = torch.nn.BatchNorm2d(out_features)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)


    def forward(self, X: torch.Tensor) -> torch.Tensor:
        X = self.conv(X)
        X = self.relu(X)
        X = self.norm(X)
        X = self.pool(X)
        return X


class Network(torch.nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        self.input_norm = torch.nn.BatchNorm2d(1, affine=False)
        self.layer1 = conv_layer(in_features=1, out_features=8)
        self.layer2 = conv_layer(in_features=8, out_features=16)
        self.layer3 = conv_layer(in_features=16, out_features=32)
        self.layer4 = conv_layer(in_features=32, out_features=64)
        self.layer5 = conv_layer(in_features=64, out_features=128)
        self.layer6 = conv_layer(in_features=128, out_features=256)
        
        

        self.net = torch.nn.Sequential(self.layer1, self.layer2, self.layer3, 
                                       self.layer4, self.layer5, self.layer6)
            
        
        self.fc1 = torch.nn.Linear(in_features=256, out_features=128)
        self.bn1 = torch.nn.BatchNorm1d(128)
        
        self.fc2 = torch.nn.Linear(in_features=128, out_features=32)
        self.bn2 = torch.nn.BatchNorm1d(32)

        self.fc3 = torch.nn.Linear(in_features=32, out_features=8)
        self.bn3 = torch.nn.BatchNorm1d(8)

        self.fc4 = torch.nn.Linear(in_features=8, out_features=4)


        self.lin = torch.nn.Sequential(self.fc1, self.bn1, self.fc2, self.bn2,
                                       self.fc3, self.bn3, self.fc4)  


    def forward(self, X: torch.Tensor) -> torch.Tensor:
        X = self.input_norm(X)
        X = self.net(X)
        X = X.reshape(X.size(0), -1)
        X = self.lin(X)
        X = torch.nn.functional.elu(X, alpha=1.0, inplace=False)
        return X

In [ ]:
net = Network()
optimizer = optim.Adam(net.parameters(), lr=1.0E-3)

PneumoClassifier = Model(net, optimizer, RMSE, device)
PneumoClassifier.train(train_data=(data.train_inputs, data.train_outputs),
                       val_data=(data.val_inputs, data.val_outputs),
                       epochs=200, patience=20, batch_size=100)


In [ ]:
PneumoClassifier.evaluate(test_data=(data.test_inputs, data.test_outputs))

In [ ]:
PneumoClassifier.plot()
PneumoClassifier.save(path="./", checkpoint_name="module")